<a href="https://colab.research.google.com/github/jiminzzni/Semeval2022/blob/main/Semeval7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 37.3 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 3.3 MB 31.6 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import logging
logging.basicConfig(level=logging.INFO)
from tqdm import tqdm
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
model.to('cuda')

candidates = ['areas', 'points', 'colors','patterns','degrees']
true_scores = [5 , 1.5 , 1, 3.333, 4.5]

title = 'How to Take Photographs That Are the Correct Color'
section_header = 'Learning how to select the correct white balance and light temperature settings.'
sentence_prev = "1. Learn how to identify white balance settings in different situations. You know this when you see the pure white, pure blacks or pure gray on the viewfinder(if there is any item in these colors)."
sentence = "2. Become familiar with how different [MASK] of color temperature affects the colors of subjects in photographs."
sentence_after = "Color temperatures refer to the amount of red or blue in the light of the photograph's setting."
text = f"{section_header + ' ' + sentence_prev + ' ' + sentence + ' ' + sentence_after}"

# The sentence with masked word, and a list of candidate words
# return a list of probabilities of candidates, e.g. [0.1, 0.2, 0.21, 0.13, 0.12]
def predict(text, candidates):
    text = f"[CLS] {text} [SEP]"
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to('cuda')

    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]
    
    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    indexed_candidates =  tokenizer.convert_tokens_to_ids(candidates)
    candidates_weights = probs[indexed_candidates]
    top_k_weights, top_k_indices = torch.topk(probs, 10, sorted=True)
  


#     for i, pred_idx in enumerate(top_k_indices):
#         predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
#         token_weight = top_k_weights[i]
#         print(f"[MASK]: {predicted_token} | weights: {float(token_weight)}")
    
#     for i, candidate in enumerate(candidates):
#         candidate_weight = candidates_weights[i]
#         print(f"[MASK]: {candidate} | weights: {float(candidate_weight)}")


    return candidates_weights.tolist()


def batch_predict(text_list, candidate_list):
    preds = []
    for i_text, text in enumerate(tqdm(text_list)):
        pred = []
        for i_candidate, candidate in enumerate(candidate_list[i_text]):
            pred.append(predict(text,candidate))
        preds.append(pred)
        pred = []
    return preds

def preprocessing1(filepath, has_header = True):
    text_list = []
    candidates_list = []
    with open(filepath, 'r') as f:
        lines = f.readlines()        
        # data preprocessing
        for line in tqdm(lines, position = 0, leave = True):
            if has_header:
                has_header = False
                continue
            line = line.strip().split('\t')

            Id = line[0]
            pattern = line[1]
            title = line[2].strip()
            section_header = line[3].strip()
            sentence_prev = line[4].strip()
            sentence = line[5].strip()
            sentence_after = line[6].strip()
            sentence = sentence[:sentence.index('______')] + '[MASK]' + sentence[sentence.index('______')+len('______'):]
            
            sentence_after = '.' if line[6].strip() == '' and sentence[-1] != '.' else ' '+line[6]
            candidates = line[7:]
            if ' ' in candidates[0]:
                candidates = [x.split()[-1] for x in candidates]
            text = f"{title + ' ' + section_header + ' ' + sentence_prev + ' ' + sentence + sentence_after}"
            
            text_list.append(text)
            candidates_list.append(candidates)

    return text_list, candidates_list
            


DEBUG:filelock:Attempting to acquire lock 140470873316176 on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Lock 140470873316176 acquired on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140470873316176 on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Lock 140470873316176 released on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Attempting to acquire lock 140469664013072 on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Lock 140469664013072 acquired on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140469664013072 on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Lock 140469664013072 released on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Attempting to acquire lock 140469664640336 on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
DEBUG:filelock:Lock 140469664640336 acquired on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


...

DEBUG:filelock:Attempting to release lock 140469664640336 on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
DEBUG:filelock:Lock 140469664640336 released on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
DEBUG:filelock:Attempting to acquire lock 140469663859344 on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:filelock:Lock 140469663859344 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140469663859344 on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:filelock:Lock 140469663859344 released on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:filelock:Attempting to acquire lock 140469659317456 on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock
DEBUG:filelock:Lock 140469659317456 acquired on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140469659317456 on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock
DEBUG:filelock:Lock 140469659317456 released on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initiali

In [5]:
def get_labels(filepath):
    labels = []
    with open(filepath, 'r') as f:
        lines = f.readlines()
        sentence = []
        for i, line in enumerate(tqdm(lines, position = 0, leave = True)):
            line = line.strip().split('\t')
            Id = line[0]
            if line[1] == 'PLAUSIBLE':
                label = 2
            elif line[1] == 'NEUTRAL':
                label = 1
            elif line[1] == 'IMPLAUSIBLE':
                label = 0
            sentence.append(label)
            
            if i % 5 == 4:
                labels.append(sentence)
                sentence = []
    return labels 
                    
labels_train = get_labels('./data/ClarificationTask_TrainLabels_Sep23.tsv')
labels_val = get_labels('./data/ClarificationTask_DevLabels_Oct22a.tsv')

100%|██████████| 2500/2500 [00:00<00:00, 995987.84it/s]


In [6]:
text_list1, candidates_list1 = preprocessing1('./data/ClarificationTask_TrainData_Sep23.tsv')
preds_train1 = batch_predict(text_list1, candidates_list1)

100%|██████████| 3995/3995 [09:43<00:00,  6.85it/s]


In [7]:
text_val_list1, candidates_val_list1 = preprocessing1('./data/ClarificationTask_DevData_Oct22a.tsv')
preds_val1 = batch_predict(text_val_list1, candidates_val_list1)

100%|██████████| 500/500 [01:14<00:00,  6.74it/s]


In [8]:
print(labels_train)

[[2, 1, 0, 1, 1], [1, 2, 2, 2, 2], [2, 0, 2, 0, 2], [2, 1, 0, 2, 0], [2, 2, 1, 2, 0], [0, 1, 1, 2, 0], [0, 2, 1, 0, 1], [0, 2, 1, 2, 0], [1, 0, 2, 0, 2], [1, 2, 0, 2, 1], [1, 0, 1, 2, 1], [2, 2, 2, 2, 1], [0, 0, 1, 1, 1], [2, 0, 0, 1, 2], [2, 2, 1, 2, 1], [1, 2, 2, 2, 2], [0, 0, 2, 1, 2], [1, 2, 0, 0, 0], [2, 0, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 2, 2, 2], [2, 1, 0, 2, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1], [1, 1, 2, 2, 1], [2, 2, 1, 1, 2], [1, 2, 2, 2, 1], [1, 1, 0, 0, 2], [0, 1, 2, 0, 1], [1, 0, 0, 0, 2], [0, 0, 2, 2, 1], [2, 0, 2, 2, 1], [0, 2, 1, 1, 0], [1, 0, 1, 1, 2], [2, 1, 2, 0, 2], [1, 2, 0, 2, 0], [1, 2, 0, 1, 2], [0, 2, 0, 1, 1], [2, 2, 0, 2, 1], [0, 0, 2, 1, 1], [2, 2, 1, 0, 2], [2, 0, 2, 2, 1], [2, 1, 1, 2, 2], [1, 0, 0, 2, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 2], [1, 2, 2, 1, 1], [1, 1, 0, 2, 2], [2, 2, 0, 2, 1], [0, 2, 2, 1, 2], [1, 1, 2, 0, 1], [1, 0, 1, 1, 2], [1, 1, 1, 2, 0], [1, 0, 2, 0, 0], [0, 2, 2, 0, 2], [2, 0, 0, 1, 2], [1, 0, 2, 1, 2], [0, 2, 0, 1, 1], [0, 2, 0, 0, 

In [20]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

preds_train1_adj = np.array([x for sub in preds_train1 for x in sub]).reshape(-1,1)
labels_train_adj = np.array([x for sub in labels_train for x in sub])
preds_val1_adj = np.array([x for sub in preds_val1 for x in sub]).reshape(-1,1)
labels_val_adj = np.array([x for sub in labels_val for x in sub])

indices_0 = [y == 0 for y in labels_train_adj] 
indices_2 = [y == 2 for y in labels_val_adj] 

preds_train1_0 = preds_train1_adj[indices_0]
preds_train1_2 = preds_train1_adj[indices_2]

labels_train_adj = labels_train_adj[indices_train]
preds_val1_adj = preds_val1_adj[indices_0] 
labels_val_adj = labels_val_adj[indices_val]
print(np.mean(preds_train1_adj.reshape(-1)))
print(np.mean(preds_train1_adj.reshape(-1)))

print(labels_train_adj.tolist())
clf.fit(preds_train1_adj, labels_train_adj)


0.03310754306382117
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

GaussianNB(priors=None, var_smoothing=1e-09)

In [11]:
print(clf.score(preds_train1_adj, labels_train_adj))
print(clf.score(preds_val1_adj, labels_val_adj))


0.5150238039491142
0.5585804569761789


In [12]:
from collections import Counter
print(Counter(clf.predict(preds_train1_adj)))
print(Counter(clf.predict(preds_val1_adj)))


Counter({0: 11504, 2: 1309})
Counter({0: 1867, 2: 190})
